In [5]:
import json
import numpy as np
import statistics
import os
import csv
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [51]:
data = pd.read_csv('data.csv')
data

,meanWidth,meanHeight,meanLength,meanVolume,countStacking,countTurnover,boxesCount,loadingWidth,loadingHeight,loadingLength,density_percent
0,130.962963,175.966667,214.596296,5.811433e+06,270,270,270,800.0,2000.0,1200.0,81.76416
1,300.000000,250.000000,400.000000,3.000000e+07,33,0,33,800.0,2000.0,1200.0,82.50000
2,800.000000,1550.000000,1200.000000,1.488000e+09,0,0,17,2500.0,2100.0,7500.0,94.44444
3,278.787879,269.696970,415.151515,3.230303e+07,33,33,33,800.0,1650.0,1200.0,74.02778
4,800.000000,1600.000000,1200.000000,1.536000e+09,0,0,33,2490.0,2590.0,13490.0,100.00000
...,...,...,...,...,...,...,...,...,...,...,...
623,825.821429,2095.000000,1211.500000,2.099682e+09,0,0,28,2400.0,2400.0,13300.0,91.90237
624,817.677419,2095.000000,1200.000000,2.055641e+09,0,0,31,2400.0,2400.0,13300.0,97.00727
625,100.000000,100.000000,100.000000,1.000000e+06,1,1,1,2450.0,2350.0,7000.0,100.00000
626,100.000000,100.000000,100.000000,1.000000e+06,2,2,2,2450.0,2350.0,7000.0,100.00000


In [42]:
data['density_percent'].value_counts()

0    329
1    299
Name: density_percent, dtype: int64

In [7]:
real_features = ['meanWidth', 'meanHeight', 'meanLength', 'meanVolume', 
                 'countStacking', 'countTurnover', 'boxesCount', 
                 'loadingWidth', 'loadingWidth', 'loadingLength']  # вещественные признаки
target_features = 'density_percent'  # целевой признак

X, y = data[real_features], data[target_features]

n_samples, n_features = X.shape

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=34)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(np.asarray(y_train.astype(np.float32)))
y_test = torch.from_numpy(np.asarray(y_test.astype(np.float32)))

In [43]:
data_sc = sc.fit_transform(data)

In [8]:
y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

In [37]:
from torch.utils.data import Dataset, DataLoader
class dataset(Dataset):
  def __init__(self,x,y):
    self.x = torch.tensor(x,dtype=torch.float32)
    self.y = torch.tensor(y,dtype=torch.float32)
    self.length = self.x.shape[0]
 
  def __getitem__(self,idx):
    return self.x[idx],self.y[idx]
  def __len__(self):
    return self.length
trainset = dataset(X.values,y.values)
#DataLoader
trainloader = DataLoader(trainset,batch_size=32,shuffle=False)

In [47]:
class BinaryClassification(nn.Module):
    def __init__(self):
        super(BinaryClassification, self).__init__()
        # Number of input features is 12.
        self.layer_1 = nn.Linear(10, 64) 
        self.layer_2 = nn.Linear(64, 64)
        self.layer_out = nn.Linear(64, 1) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(64)
        self.batchnorm2 = nn.BatchNorm1d(64)
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.layer_out(x)
        
        return x

In [49]:
learning_rate = 0.01
epochs = 700
# Model , Optimizer, Loss
model = BinaryClassification()
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)
loss_fn = nn.BCELoss()

In [ ]:
losses = []
accur = []
for i in range(epochs):
  for j,(x_train,y_train) in enumerate(trainloader):
    
    #calculate output
    output = model(x_train)
 
    #calculate loss
    loss = loss_fn(output,y_train.reshape(-1,1))
 
    #accuracy
    predicted = model(torch.tensor(x,dtype=torch.float32))
    acc = (predicted.reshape(-1).detach().numpy().round() == y).mean()
    #backprop
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  if i%50 == 0:
    losses.append(loss)
    accur.append(acc)
    print("epoch {}\tloss : {}\t accuracy : {}".format(i,loss,acc))

In [53]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Определение нейросети
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(10, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 1)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Определение класса набора данных
class CustomDataset(Dataset):
    def __init__(self, data):
        self.inputs = torch.tensor(data[:, :-1], dtype=torch.float32)
        self.targets = torch.tensor(data[:, -1], dtype=torch.float32)
    
    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, idx):
        return self.inputs[idx], self.targets[idx]

# Создание обучающего и тестового набора данных
data = pd.read_csv('data.csv')

data = torch.tensor(data.values)

train_dataset = CustomDataset(data[:500])
test_dataset = CustomDataset(data[500:])

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32)

# Инициализация нейросети и оптимизатора
net = Net()
optimizer = optim.SGD(net.parameters(), lr=0.01)
loss_function = nn.MSELoss()

# Обучение нейросети
num_epochs = 10

for epoch in range(num_epochs):
    net.train()  # Переключение в режим обучения
    train_loss = 0.0

    for inputs, targets in train_dataloader:
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = loss_function(outputs.squeeze(), targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    # Вычисление и вывод среднего значения функции потерь на обучающем наборе данных
    train_loss /= len(train_dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss}")

# Пример использования обученной нейросети для предсказания целевого признака
input_example = torch.tensor([[130.962963, 175.966667, 214.596296, 5.811433e+06, 270, 270, 270, 800.0, 2000.0, 1200.0]], dtype=torch.float32)
predicted_value = net(input_example).item()
print(f"Predicted Value: {predicted_value}")


<ipython-input-53-7f4d34f623a0>:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.inputs = torch.tensor(data[:, :-1], dtype=torch.float32)
<ipython-input-53-7f4d34f623a0>:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.targets = torch.tensor(data[:, -1], dtype=torch.float32)


Epoch 1/10, Train Loss: nan
Epoch 2/10, Train Loss: nan
Epoch 3/10, Train Loss: nan
Epoch 4/10, Train Loss: nan
Epoch 5/10, Train Loss: nan
Epoch 6/10, Train Loss: nan
Epoch 7/10, Train Loss: nan
Epoch 8/10, Train Loss: nan
Epoch 9/10, Train Loss: nan
Epoch 10/10, Train Loss: nan
Predicted Value: nan


In [54]:
data[:500]

tensor([[  130.9630,   175.9667,   214.5963,  ...,  2000.0000,  1200.0000,
            81.7642],
        [  300.0000,   250.0000,   400.0000,  ...,  2000.0000,  1200.0000,
            82.5000],
        [  800.0000,  1550.0000,  1200.0000,  ...,  2100.0000,  7500.0000,
            94.4444],
        ...,
        [  620.0714,   467.3214,  1672.9643,  ...,  2590.0000, 13490.0000,
            38.5464],
        [  620.0714,   467.3214,  1672.9643,  ...,  2590.0000, 13490.0000,
            72.0727],
        [  620.0714,   467.3214,  1672.9643,  ...,  2590.0000, 13490.0000,
            65.4745]], dtype=torch.float64)